In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import Dash, dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import CRUD file for handling database functionality
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
db = AnimalShelter("Employee", "SNHU1234")

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['animal_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Accessing and reading logo for client 
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
#encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# variable for html of client's image
#logo_img = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(
        #logo_img,  # Displaying the image at the top of the layout
        #style={'text-align': 'center'} # Centering the image
    #),
    html.A("Client Home Page", href="www.snhu.edu"),
    html.Center(html.B(html.H1('Graziosalvare Dashboard - Wes Roberts'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(id='filter-type',
                       options=[
                           {'label': 'Standard', 'value': 'standard'},
                           {'label': 'Water Rescue', 'value': 'water_rescue'},
                           {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
                           {'label': 'Disaster or Individual Tracking', 'value': 'disaster_tracking'},
                       ],
                       value='standard',
                       inline=True
                      )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                         
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
    html.Div(
        id='graph-id',
        className='col s12 m6',

            ),
    html.Div(
        id='map-id',
        className='col s12 m6',
        ),
    html.H6(children="Wesley Roberts' Authenticator"),
    html.Div(id='breed-count', children=[])
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_selected_filter(filter_type):
    df = pd.DataFrame.from_records(db.read({}))
    # Logic for handling conditions of the RadioItems selections
    if filter_type == 'standard':
        df = pd.DataFrame.from_records(db.read({})) #Default value
    elif filter_type == 'water_rescue':
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog",
                                                "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix"]},
                                                "sex_upon_outcome": "Intact Female",
                                                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}))
    elif filter_type == 'mountain_rescue':
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog",
                                                "breed": {"$in": ["German Shepard Mix", "Alaskan Malamute Mix", "Old English Sheepdog", "Siberian Husky Mix", "Rottweiler Mix"]},
                                                "sex_upon_outcome": "Intact Male",
                                                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}))
    elif filter_type == 'disaster_tracking':
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog",
                                                "breed": {"$in": ["Doberman Pinsch", "Doberman Pinsch Mix", "German Shepherd Mix", "Golden Retriever Mix", "Bloodhound", "Bloodhound Mix", "Rottweiler Mix"]},
                                                "sex_upon_outcome": "Intact Male",
                                                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}))
    else:
        df = pd.DataFrame.from_records(db.read({}))  # Default to an empty query for unknown filter_type values
    df.drop(columns=['animal_id'], inplace=True)
    
    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Check if viewData is not None
    if viewData is not None:
        # Convert viewData to a DataFrame (different variable than others, because it was causing errors for other charts)
        dfff = pd.DataFrame(viewData)

        # Calculate percentages based on the 'breed' column - setting up avoiding displaying any value less than 1%
        breed_counts = dfff['breed'].value_counts(normalize=True) * 100

        # Filter to only display breeds with more than 1% on pie chart
        filtered_data = dfff[dfff['breed'].isin(breed_counts[breed_counts > 1].index)]

        # Generate a pie chart
        pie_chart = dcc.Graph(
            figure=px.pie(filtered_data, names='breed', title='Preferred Animals')
        )

        # Return the pie chart as a list
        return [pie_chart]

    # If viewData is None, return an empty list to avoid raise condition error
    return []
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    
# Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 12 and 13 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 8 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,11],dff.iloc[row,12]],
              children=[
              dl.Tooltip(dff.iloc[row,14]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row,14])
             ])
          ]) if not dff.empty else dl.Marker(position=[0, 0]) # Handles the raise condition error
       ])
    ]


print("This reached here")
app.run_server(debug=True)


This reached here


C:\Python312\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/


In [1]:
from Module4 import AnimalShelter

In [2]:
db = AnimalShelter("", "")

In [3]:
db.read()

In [4]:
import pandas as pd

In [5]:
df = pd.DataFrame.from_records(db.read())

In [6]:
df.head()

,,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,outcome_subtype,outcome_type,sex_upon_outcome,location_lat,location_long,age_upon_outcome_in_weeks,name
0,1,3 years,A746874,Cat,Domestic Shorthair Mix,Black/White,4/10/2014,4/11/2017 9:00,2017-04-11T09:00:00,SCRP,Transfer,Neutered Male,30.506658,-97.340878,156.767857,NaN
1,2,1 year,A725717,Cat,Domestic Shorthair Mix,Silver Tabby,5/2/2015,5/6/2016 10:49,2016-05-06T10:49:00,SCRP,Transfer,Spayed Female,30.652598,-97.741996,52.921528,NaN
2,3,2 years,A716330,Dog,Chihuahua Shorthair Mix,Brown/White,11/18/2013,12/28/2015 18:43,2015-12-28T18:43:00,NaN,Adoption,Neutered Male,30.759575,-97.552375,110.111409,Frank
3,4,7 months,A733653,Cat,Siamese Mix,Seal Point,1/25/2016,8/27/2016 18:11,2016-08-27T18:11:00,NaN,Adoption,Intact Female,30.318806,-97.724038,30.822520,Kitty
4,5,2 years,A691584,Dog,Labrador Retriever Mix,Tan/White,11/6/2012,5/30/2015 13:48,2015-05-30T13:48:00,NaN,Return to Owner,Neutered Male,30.710482,-97.562297,133.653571,Luke


In [7]:
df.count()

                             10000
age_upon_outcome             10000
animal_id                    10000
animal_type                  10000
breed                        10000
color                        10000
date_of_birth                10000
datetime                     10000
monthyear                    10000
outcome_subtype               4599
outcome_type                  9998
sex_upon_outcome             10000
location_lat                 10000
location_long                10000
age_upon_outcome_in_weeks    10000
name                          6956
dtype: int64